In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [2]:
input_text = au.read_txt_file_lines('input.txt')
n_rows = len(input_text)
print(f"n_rows = {n_rows}")
brick_coords = collections.namedtuple('brick_coords', 'x_1 y_1 z_1 x_2 y_2 z_2 xy_footprint')
list_of_brick_coords = []
for l in input_text:
    coords = l.split('~')
    x_1, y_1, z_1 = [int(x) for x in coords[0].split(',')]
    x_2, y_2, z_2 = [int(x) for x in coords[1].split(',')]
    assert z_1 <= z_2, l
    assert y_1 <= y_2, l
    assert x_1 <= x_2, l
    assert int(x_1 == x_2) + int(y_1 == y_2) + int(z_1 == z_2) >= 2, l
    if x_1 == x_2 and y_1 == y_2:
        xy_footprint = set((x_1, y_1) for _ in range(2))  # do this trick to make it a set of two equal numbers inside tuple 
    elif x_1 == x_2 and y_1 != y_2:
        xy_footprint = set((x_1, y) for y in range(y_1, y_2 + 1))
    elif x_1 != x_2 and y_1 == y_2:
        xy_footprint = set((x, y_1) for x in range(x_1, x_2 + 1))
    else:
        assert False, l
    list_of_brick_coords.append(brick_coords(x_1, y_1, z_1, x_2, y_2, z_2, xy_footprint))
    

n_rows = 1229


In [3]:
## sort by z_max then z_min:
list_of_brick_coords = sorted(list_of_brick_coords, key=lambda x: x.z_2)
list_of_brick_coords = sorted(list_of_brick_coords, key=lambda x: x.z_1)
list_of_brick_coords[:10]

[brick_coords(x_1=1, y_1=2, z_1=1, x_2=1, y_2=3, z_2=1, xy_footprint={(1, 2), (1, 3)}),
 brick_coords(x_1=5, y_1=5, z_1=1, x_2=5, y_2=7, z_2=1, xy_footprint={(5, 5), (5, 6), (5, 7)}),
 brick_coords(x_1=7, y_1=5, z_1=1, x_2=7, y_2=9, z_2=1, xy_footprint={(7, 7), (7, 9), (7, 6), (7, 5), (7, 8)}),
 brick_coords(x_1=0, y_1=4, z_1=1, x_2=1, y_2=4, z_2=1, xy_footprint={(0, 4), (1, 4)}),
 brick_coords(x_1=4, y_1=2, z_1=1, x_2=6, y_2=2, z_2=1, xy_footprint={(6, 2), (4, 2), (5, 2)}),
 brick_coords(x_1=8, y_1=5, z_1=1, x_2=8, y_2=8, z_2=1, xy_footprint={(8, 7), (8, 8), (8, 5), (8, 6)}),
 brick_coords(x_1=1, y_1=6, z_1=1, x_2=4, y_2=6, z_2=1, xy_footprint={(1, 6), (4, 6), (2, 6), (3, 6)}),
 brick_coords(x_1=7, y_1=2, z_1=1, x_2=7, y_2=4, z_2=1, xy_footprint={(7, 4), (7, 2), (7, 3)}),
 brick_coords(x_1=2, y_1=0, z_1=1, x_2=2, y_2=0, z_2=1, xy_footprint={(2, 0)}),
 brick_coords(x_1=1, y_1=5, z_1=1, x_2=2, y_2=5, z_2=1, xy_footprint={(2, 5), (1, 5)})]

In [4]:
def check_for_other_bricks_below(focal_brick, list_bricks):
    curr_z = focal_brick.z_1
    if curr_z == 1:
        # print('reached bottom')
        return True
    for brick in list_bricks:
        if brick.z_2 == curr_z - 1:
            if len(focal_brick.xy_footprint.intersection(brick.xy_footprint)) > 0:
                # print(f"brick {brick} is below {focal_brick}")
                return True
    return False

def check_for_all_other_bricks_below(focal_brick, list_bricks):
    curr_z = focal_brick.z_1
    if curr_z == 1:
        assert False, 'not expecting to reach bottom'
        return []
    list_bricks_below = []
    for i_brick, brick in enumerate(list_bricks):
        if brick.z_2 == curr_z - 1:
            if len(focal_brick.xy_footprint.intersection(brick.xy_footprint)) > 0:
                list_bricks_below.append(i_brick)
    return list_bricks_below
    


In [5]:
def settle_bricks(list_of_brick_coords, verbose=1):
    for ii, brick in enumerate(list_of_brick_coords):
        if brick.z_1 == 1:
            if verbose > 0:
                print(f"brick {ii} is on the bottom")
            continue

        tmp_brick = copy.deepcopy(brick)
        while not check_for_other_bricks_below(tmp_brick, list_of_brick_coords[:ii]):
            # print('fall')
            tmp_brick = brick_coords(tmp_brick.x_1, tmp_brick.y_1, tmp_brick.z_1 - 1, tmp_brick.x_2, tmp_brick.y_2, tmp_brick.z_2 - 1, tmp_brick.xy_footprint)
            if verbose > 0:
                print(f"tmp_brick = {tmp_brick}")
        list_of_brick_coords[ii] = tmp_brick  
    return list_of_brick_coords

settled_list_of_brick_coords = settle_bricks(list_of_brick_coords, verbose=0)

print('BRICKS SETTLED')
    

BRICKS SETTLED


In [6]:
list_unique_supporting_bricks = []
for ii, brick in enumerate(settled_list_of_brick_coords):
    if brick.z_1 == 1:
        # print(f"brick {ii} is on the bottom")
        continue 

    list_bricks_below = check_for_all_other_bricks_below(brick, settled_list_of_brick_coords)
    assert len(list_bricks_below) > 0, f"brick {ii} has no bricks below it"
    # print(f"brick {ii} has {len(list_bricks_below)} bricks below it")
    if len(list_bricks_below) == 1:
        list_unique_supporting_bricks.append(list_bricks_below[0])
    
n_unique_supporting_bricks = len(set(list_unique_supporting_bricks))
# assert len(list_unique_supporting_bricks) == len(set(list_unique_supporting_bricks)), f' {len(list_unique_supporting_bricks)} != {len(set(list_unique_supporting_bricks))}'
len(settled_list_of_brick_coords) - np.unique(list_unique_supporting_bricks).shape[0]   

437

## part 2

In [7]:
n_new_falls = 0
    
for ii, brick in tqdm(enumerate(settled_list_of_brick_coords)):
    new_list_of_bricks_without_brick = settled_list_of_brick_coords[:ii] + settled_list_of_brick_coords[ii + 1:]
    new_settled_list = settle_bricks(copy.deepcopy(new_list_of_bricks_without_brick), verbose=0)
    ## see how many bricks changed z_1:
    for jj, new_brick in enumerate(new_settled_list):
        if new_brick.z_1 != new_list_of_bricks_without_brick[jj].z_1:
            n_new_falls += 1
    # print(ii, n_new_falls)

print(n_new_falls)


1229it [00:52, 23.57it/s]

42561
